# Imports

In [12]:
import numpy as np
import h5py as h5

import os, glob, fnmatch, imp
from collections import OrderedDict

import rec_new; imp.reload(rec_new)
from rec_new import rec, rec_try

import pandas as pd

# Base Path

In [3]:
path = '/local/data/2019-04/Toney/'

## Get al Good Scans

In [17]:
h5files = fnmatch.filter(os.listdir(path), '*.h5')
h5files.sort()

goodFiles, goodScans = [], []
CountTimes = OrderedDict()

for file in h5files[:]:
    with h5.File(os.path.join(path, file)) as ff:
        if ff['exchange']['data'].shape[0] != 1500:
            continue

        goodFiles.append(file); goodScans.append(file[:-3])
        countTime = ff['measurement']['instrument']['detector']['exposure_time'].value[0]
        CountTimes[file] = np.round(countTime, 3)
        
df = pd.DataFrame.from_dict(CountTimes, orient='index', columns=['countTime'])

## Trial reconstructions to get COR

In [18]:
finished_scans = next(os.walk(os.path.join(path, 'centers')))[1]
finished_scans = [(scan+'.h5') for scan in finished_scans if scan in goodScans]
finished_scans.sort()

for scan, countTime in CountTimes.items():
    if (scan in finished_scans):
        continue
   
    dark_scan = None
    if (countTime == 0.5) & (int(scan[:3]) < 80):
        dark_scan = '131_white_dark_0p5s.h5'  #'133_white_dark_0p2s.h5'
        
    print(f'\n{scan}')
    
    h5name = os.path.join(path, scan)
    dark_file = os.path.join(path, dark_scan) if dark_scan else None
        
    rec(h5name, rec_type='try', rot_center=1220, algorithm='astra_fbp', center_search_width=30, dark_file=dark_file)

# Full Reconstructions

## Get Center Values from CSV File

In [19]:
center_fname = 'Sample and scans of APS 2BM, Apr 2-4 - Sheet2.csv'
center_file = os.path.join(path, center_fname)
df_Centers = pd.read_csv(center_file, index_col='Scan')

In [22]:
for scan in df_Centers.index[:]:
    if df_Centers.Count_Time[scan] != 0.5:
        continue
    
    print(f'\nScan {scan}')
    
    dark_scan = None
    if (int(scan[:3]) < 80):
        print('Using 131_white_dark_0p5s.h5 for white/dark images')
        dark_scan = '131_white_dark_0p5s.h5'  #'133_white_dark_0p2s.h5'

    h5name = os.path.join(path, scan)
    dark_file = os.path.join(path, dark_scan) if dark_scan else None
    
    rot_center = df_Centers.Center[scan]
    rec(h5name, rec_type='full', rot_center=rot_center, algorithm='gridrec', dark_file=dark_file)


Scan 019_CFF_B36A_P98_35keV.h5
Using 131_white_dark_0p5s.h5 for white/dark images
Reconstructing a single file
Reconstructing [2048] slices from slice [0] to [2048] in [6] chunks of [341] slices each

  -- chunk # 1

  --------> [0, 341]
Raw data:  /local/data/2019-04/Toney/019_CFF_B36A_P98_35keV.h5
Center:  1224.5
gridrec
Algorithm:  gridrec
Reconstructions:  /local/data/2019-04/Toney/019_CFF_B36A_P98_35keV_full_rec//gridrec/recon

  -- chunk # 2

  --------> [341, 682]
Raw data:  /local/data/2019-04/Toney/019_CFF_B36A_P98_35keV.h5
Center:  1224.5
gridrec
Algorithm:  gridrec
Reconstructions:  /local/data/2019-04/Toney/019_CFF_B36A_P98_35keV_full_rec//gridrec/recon

  -- chunk # 3

  --------> [682, 1024]


OSError: [Errno 28] No space left on device

In [ ]:
for scan in df_Centers.index[3:]:
    if df_Centers.Count_Time[scan] == 0.5:
        continue
    
    print(f'\nScan {scan}')
    
    h5name = os.path.join(path, scan)
    rot_center = df_Centers.Center[scan]
    
    if df_Centers.Count_Time[scan] == 0.2:
        rec(h5name, rec_type='full', rot_center=rot_center, algorithm='gridrec')
    else:
        rec(h5name, rec_type='full', rot_center=rot_center, algorithm='astra_fbp')